In [ ]:
import h5py
import numpy as np
import tensorflow.keras as Keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tvsumh5datasets = '/content/drive/MyDrive/tvsum (2).h5'

In [ ]:
class DatasetLoader():
  def __init__(self, dataset_path):
    self.dataset = dataset_path
  def __len__(self):
    return len(self.dataset)
  def __getitem__(self,index_of_vidoes):
    index_of_vidoes = index_of_vidoes +1
    video = self.dataset['video_'+str(index_of_vidoes)]
    feature = video['feature'][:]
    label = video['label'][:]

    feature = np.array(feature)
    label = np.array(label)


    return feature, label, index_of_vidoes

In [ ]:
class DatasetBatchCreatorForTraining(Sequence):
  def __init__(self, dataset_path, batch_size, shuffle=False):
    self.dataset = dataset_path
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indices = np.arange(len(self.dataset))
    if self.shuffle:
      np.random.shuffle(self.indices)

  def __len__(self):
    length_of_dataset = len(self.dataset)/self.batch_size
    length_of_dataset = np.floor(length_of_dataset)
    length_of_dataset = int(length_of_dataset)
    return length_of_dataset

  def __getitem__(self,index):
    indexes = self.indices[index * self.batch_size : (index+1) * self.batch_size]
    feature = np.empty((self.batch_size,320,1024)) #
    label = np.empty((self.batch_size,320,1)) #

    for i in range(len(indexes)):
      feature[i] = np.array(self.dataset[indexes[i]][0])
      label[i] = np.array(self.dataset[indexes[i]][1]).reshape(-1,1)

    return feature, label



In [ ]:
datasetLoader = DatasetLoader(h5py.File(tvsumh5datasets))
X, y = train_test_split(datasetLoader, test_size=0.2)

In [ ]:
X_with_batch_size = DatasetBatchCreatorForTraining(X,8)

In [ ]:
model_one = Sequential()
model_one.add(LSTM(128, return_sequences=True, input_shape=(320, 1024)))
model_one.add(LSTM(256, return_sequences=True))
model_one.add(Dense(256, activation='relu'))
model_one.add(Dense(128, activation='relu'))
model_one.add(TimeDistributed(Dense(1, activation='sigmoid')))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# model 2
model2 = Sequential()
model2.add(LSTM(128, return_sequences=True, input_shape=(320, 1024)))
model2.add(BatchNormalization())
model2.add(LSTM(256, return_sequences=True))
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(TimeDistributed(Dense(1, activation='sigmoid')))

In [ ]:
model_one.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [ ]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_one.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                       │ (None, 320, 128)            │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 320, 256)            │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_9 (TimeDistributed) │ (None, 320, 1)              │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 984,833 (3.76 MB)

 Trainable params: 984,833 (3.76 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model2.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_20 (LSTM)                       │ (None, 320, 128)            │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 320, 256)            │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_10                  │ (None, 320, 1)              │             257 │
│ (TimeDistributed)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 984,833 (3.76 MB)

 Trainable params: 984,833 (3.76 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stopping = EarlyStopping(patience=1, restore_best_weights=True)


In [ ]:
model_one.fit(X_with_batch_size, epochs=10)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - binary_accuracy: 0.5624 - loss: 0.5918
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - binary_accuracy: 0.8176 - loss: 0.4845
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - binary_accuracy: 0.8180 - loss: 0.4753
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - binary_accuracy: 0.8145 - loss: 0.4819
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - binary_accuracy: 0.8179 - loss: 0.4756
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - binary_accuracy: 0.8176 - loss: 0.4746
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - binary_accuracy: 0.8128 - loss: 0.4822
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - binary_accuracy: 0.8163 - loss: 0.4767
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - binary_accuracy: 0.8133 - loss: 0.4815
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - binary_accuracy: 0.8123 - loss: 0.4826


In [ ]:
model2.fit(X_with_batch_size, epochs=10, callbacks=[early_stopping])

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - accuracy: 0.5474 - loss: 0.6067
Epoch 2/10
2/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.8214 - loss: 0.4748

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8166 - loss: 0.4846
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.8109 - loss: 0.4879
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8184 - loss: 0.4758
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.8155 - loss: 0.4777
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8140 - loss: 0.4819
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8130 - loss: 0.4826
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8167 - loss: 0.4757
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.8139 - loss: 0.4799
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.8135 - loss: 0.4802


In [ ]:
model_one.save('/content/drive/MyDrive/model_one.keras')

In [ ]:
model2.save('/content/drive/MyDrive/model2.keras')

In [ ]:
class EvaluationKeys():
  def __init__(self):
    pass

  def knapsack(self,s, w, c): #shot, weights, capacity


    shot = len(s) + 1 #number of shots
    cap = c + 1 #capacity threshold

      #matching the modified size by adding 0 at 0th index
    s = np.r_[[0], s] #adding 0 at 0th index (concatinating)
    w = np.r_[[0], w] #adding 0 at 0th index (concatinating)

      #Creating and Filling Dynamic Programming Table with zeros in shot x cap dimensions
    dp = [] #creating empty list or table
    for j in range(shot):
      dp.append([]) #s+1 rows
      for i in range(cap):
        dp[j].append(0) #c+1 columns

      #Started filling values from (2nd row, 2nd column) till (shot X cap) and keeping the values for 0th indexes as 0
      #following dynamic programming approach to fill values
    for i in range(1,shot):
      for j in range(1,cap):
        if w[i] <= j:
          dp[i][j] = max(s[i] + dp[i-1][j-w[i]], dp[i-1][j])
        else:
          dp[i][j] = dp[i-1][j]
    choice = []
    i = shot - 1
    j = cap - 1
    while i > 0 and j > 0:
        if dp[i][j] != dp[i-1][j]: #starting from last element and going further
            choice.append(i-1)
            j = j - w[i]
            i = i - 1
        else:
            i = i - 1

    return dp[shot-1][cap-1], choice




  def eval_metrics(self,y_pred, y_true):
    '''Returns precision, recall and f1-score of given prediction and true value'''
    true_positive = np.sum(y_pred * y_true)
    avoid_zero_denominator = 1e-8
    precision = true_positive / (np.sum(y_pred) + avoid_zero_denominator) # true positive + false positive
    recall = true_positive / (np.sum(y_true) + avoid_zero_denominator) # true positive + false negative
    if precision == 0 and recall == 0:
        f1score = 0
    else:
        f1score = 2 * precision * recall / (precision + recall) # F1score = 2*P*R/(P+R)

    return [precision, recall, f1score]



  def upsample(self,down_arr, vidlen):
    up_arr = np.zeros(vidlen)
    ratio = vidlen // 320
    l = (vidlen - ratio * 320) // 2
    i = 0
    while i < 320:
        up_arr[l:l+ratio] = np.ones(ratio, dtype = int) * down_arr[0][i]
        l += ratio
        i += 1

    return up_arr

  def select_keyshots(self,video_info, pred_score):
    vidlen = video_info['length'][()] # Getting video length
    cps = video_info['change_points'][:] # Getting change points (shot changes)
    weight = video_info['n_frame_per_seg'][:] # Getting number of frames per shot info
    pred_score = np.array(pred_score)
    pred_score = self.upsample(pred_score, vidlen)
    pred_value = np.array([pred_score[cp[0]:cp[1]].mean() for cp in cps])
    _, selected = self.knapsack(pred_value, weight, int(0.15 * vidlen)) # Knapsacking 15% of the video as summary
    selected = selected[::-1]
    key_labels = np.zeros((vidlen,))
    for i in selected:
        key_labels[cps[i][0]:cps[i][1]] = 1

    return pred_score.tolist(), selected, key_labels.tolist()

In [ ]:
import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model
import matplotlib.pyplot as plt

class ModelEvaluator(object):


  def __init__(self):
    self.train_metrics = {'accuracy':[],
                          'loss':[]}
    self.test_metrics = {'accuracy':[],
                         'loss':[]}
    self.train_eval_arr = []
    self.test_eval_arr = []
    self.eval_arr = []
  # evaluate model
  def evaluate_model(self, dataset_type,model,dataset_name):
      '''Evaluates the model for given epoch on test dataset'''
      out_dict = {}
      evaluationKeys = EvaluationKeys()
      table = PrettyTable()
      table.field_names = ['ID', 'Precision', 'Recall', 'F-Score']
      table.float_format = '1.5'


      with h5py.File(tvsumh5datasets) as data_file:
          for feature, _, index in tqdm(dataset_type, desc = 'Evaluate', ncols = 90, leave = False):

              pred_score = model.predict(feature.reshape(-1,320,1024)) # Predicting for 'index' video
              video_info = data_file['video_'+str(index)]
              pred_score, pred_selected, pred_summary = evaluationKeys.select_keyshots(video_info, pred_score) # Getting predictions of shot level
              true_summary_arr = video_info['user_summary'][:] # True summary of the video
              eval_res = [evaluationKeys.eval_metrics(pred_summary, true_summary) for true_summary in true_summary_arr] # Getting evaluation metrics of that particular video
              eval_res = np.mean(eval_res, axis = 0).tolist() # Averaging out the metrics

              if dataset_name == 'train_datasets':
                self.train_eval_arr.append(eval_res)
              else:
                self.test_eval_arr.append(eval_res)


              self.eval_arr.append(eval_res)
              table.add_row([index] + eval_res)

              out_dict[str(index)] = {
              'pred_score' : pred_score,
              'pred_selected' : pred_selected,
              'pred_summary' : pred_summary
              }
      eval_mean = np.mean(self.eval_arr, axis = 0).tolist()
      table.add_row(['mean'] + eval_mean)
      tqdm.write(str(table))




In [ ]:
modelEvaluator =  ModelEvaluator()
modelEvaluator.evaluate_model(y,model_one,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Evaluate:  10%|████▍                                       | 1/10 [00:03<00:28,  3.19s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:03<00:11,  1.39s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:04<00:00,  3.93it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  3.85it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  17  |  0.60829  | 0.59506 | 0.60159 |
|  47  |  0.53523  | 0.52738 | 0.53126 |
|  46  |  0.57409  | 0.56582 | 0.56993 |
|  24  |  0.55695  | 0.54716 | 0.55199 |
|  48  |  0.37039  | 0.36146 | 0.36584 |
|  35  |  0.60926  | 0.60878 | 0.60899 |
|  40  |  0.57616  | 0.56916 | 0.57264 |
|  8   |  0.41853  | 0.41456 | 0.41653 |
|  3   |  0.57035  | 0.56276 | 0.56653 |
|  13  |  0.45906  | 0.44747 | 0.45316 |
| mean |  0.52783  | 0.51996 | 0.52385 |
+------+-----------+---------+---------+


In [ ]:
modelEvaluator.evaluate_model(y,model2,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Evaluate:  10%|████▍                                       | 1/10 [00:03<00:28,  3.21s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:03<00:11,  1.40s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:04<00:00,  3.95it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  3.88it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  17  |  0.58822  | 0.57216 | 0.58007 |
|  47  |  0.49357  | 0.48393 | 0.48868 |
|  46  |  0.57601  | 0.56845 | 0.57220 |
|  24  |  0.53750  | 0.53421 | 0.53583 |
|  48  |  0.42554  | 0.41355 | 0.41942 |
|  35  |  0.60926  | 0.60878 | 0.60899 |
|  40  |  0.56138  | 0.55422 | 0.55777 |
|  8   |  0.42752  | 0.42363 | 0.42556 |
|  3   |  0.52827  | 0.52126 | 0.52474 |
|  13  |  0.55472  | 0.54167 | 0.54807 |
| mean |  0.52901  | 0.52107 | 0.52499 |
+------+-----------+---------+---------+


In [ ]:
import tensorflow as tf

model3 = tf.keras.models.load_model('/content/drive/MyDrive/model3.h5')
model4 = tf.keras.models.load_model('/content/drive/MyDrive/model4.h5')

print("Model 3 loaded successfully.")
print("Model 4 loaded successfully.")

Model 3 loaded successfully.
Model 4 loaded successfully.


In [ ]:

# class ModelEvaluatorED(object):

#     def __init__(self):
#         self.train_metrics = {'accuracy': [], 'loss': []}
#         self.test_metrics = {'accuracy': [], 'loss': []}
#         self.train_eval_arr = []
#         self.test_eval_arr = []
#         self.eval_arr = []

#     # Evaluate encoder-decoder model
#     def evaluate_encoder_decoder_model(self, dataset_type, model, dataset_name):
#         '''Evaluates the model for given epoch on test dataset'''
#         out_dict = {}
#         evaluationKeys = EvaluationKeys()
#         table = PrettyTable()
#         table.field_names = ['ID', 'Precision', 'Recall', 'F-Score']
#         table.float_format = '1.5'

#         with h5py.File(tvsumh5datasets) as data_file:
#             for feature, _, index in tqdm(dataset_type, desc='Evaluate', ncols=90, leave=False):
#                 # Print the shape of feature for debugging
#                 print(f"Feature shape for index {index}: {feature.shape}")

#                 # Check if feature has the expected dimensions
#                 if len(feature.shape) == 3:
#                     # Prepare decoder inputs for evaluation
#                     decoder_input = np.zeros((1, feature.shape[0], feature.shape[1], 1024))
#                     decoder_input[:, 1:, :] = feature[:, :-1, :]

#                     # X_val_decoder = np.zeros((y_val.shape[0], y_val.shape[1], 1024))
#                     # X_val_decoder[:, 1:, :] = X_val[:, :-1, :]

#                     # Predicting for 'index' video
#                     pred_score = model.predict([feature.reshape(-1, 320, 1024), decoder_input])

#                     video_info = data_file['video_' + str(index)]
#                     pred_score, pred_selected, pred_summary = evaluationKeys.select_keyshots(video_info, pred_score)  # Getting predictions of shot level
#                     true_summary_arr = video_info['user_summary'][:]  # True summary of the video
#                     eval_res = [evaluationKeys.eval_metrics(pred_summary, true_summary) for true_summary in true_summary_arr]  # Getting evaluation metrics of that particular video
#                     eval_res = np.mean(eval_res, axis=0).tolist()  # Averaging out the metrics

#                     if dataset_name == 'train_datasets':
#                         self.train_eval_arr.append(eval_res)
#                     else:
#                         self.test_eval_arr.append(eval_res)

#                     self.eval_arr.append(eval_res)
#                     table.add_row([index] + eval_res)

#                     out_dict[str(index)] = {
#                         'pred_score': pred_score,
#                         'pred_selected': pred_selected,
#                         'pred_summary': pred_summary
#                     }
#                 else:
#                     print(f"Skipping index {index} due to unexpected feature shape: {feature.shape}")

#         eval_mean = np.mean(self.eval_arr, axis=0)
#         if isinstance(eval_mean, float):
#             eval_mean = [eval_mean]
#         else:
#             eval_mean = eval_mean.tolist()
#         table.add_row(['mean'] + [f'{x:.5f}' for x in eval_mean])
#         tqdm.write(str(table))

In [ ]:
# Instantiate the ModelEvaluatorED class
# modelEvaluatorED = ModelEvaluatorED()



In [ ]:
# Call the evaluate_encoder_decoder_model method
# modelEvaluatorED.evaluate_encoder_decoder_model(y, model3, "test_datasets")

In [ ]:
modelEvaluator =  ModelEvaluator()
modelEvaluator.evaluate_model(y,model_one,"test_datasets")

In [ ]:
data_file = h5py.File("/content/test_data_.h5")
datasetLoader = DatasetLoader(data_file)


In [ ]:
video = data_file['video_'+str(1)]
feature = video['feature'][:]
feature = np.array(feature)


dataset = feature, 1

In [ ]:
predicted_score = loaded_model.predict(feature.reshape(-1,320,1024))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


In [ ]:
pred_score, pred_selected, pred_summary = evaluationKeys.select_keyshots(video, predicted_score)

In [ ]:
# pred_score

In [ ]:
# pred_selected

In [ ]:
# pred_summary

In [ ]:
import cv2

In [ ]:


key_labels = pred_summary

    # --- Create Summary Video ---
video = cv2.VideoCapture("/content/drive/MyDrive/XzYM3PfTM4w.mp4")  # Reopen
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Or other codec
fps = video.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("summary.mp4", fourcc, fps, (int(video.get(3)), int(video.get(4))))

frame_index = 0
success, frame = video.read()
while success:
    if key_labels[frame_index] == 1:
        out.write(frame)
    frame_index += 1
    success, frame = video.read()

video.release()
out.release()



In [ ]:
# encoder decoder
import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model

encoder_inputs = Input(shape = (320, 1024))

encoder_BidirectionalLSTM = Bidirectional(LSTM(128, return_sequences = True, return_state = True)) # Encoder Layer
encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM(encoder_inputs)
sh = Concatenate()([fh, bh])
ch = Concatenate()([fc, bc])
encoder_states = [sh, ch]

decoder_LSTM = LSTM(256, return_sequences = True, dropout = 0.01, recurrent_dropout = 0.01) # Decoder Layer
decoder_out = decoder_LSTM(encoder_out, initial_state = encoder_states)

attn_layer = Attention(name="Attention_Layer") # Attention Layer
attn_out =  attn_layer([encoder_out, decoder_out])

decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_out, attn_out]) # Concatenating outputs of attention and decoder layer

dense = TimeDistributed(Dense(1, activation = 'sigmoid')) # Output layer
decoder_pred = dense(decoder_concat_input)

model_ed = Model(inputs = encoder_inputs, outputs = decoder_pred) # Model assembling

opt = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1 = 0.8, beta_2 = 0.8) # Adam Optimizer
#opt = tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model_ed.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy']) # Model Compiling
model_ed.summary()



Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 320, 1024)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ [(None, 320, 256),     │      1,180,672 │ input_layer_2[0][0]    │
│ (Bidirectional)           │ (None, 128), (None,    │                │                        │
│                           │ 128), (None, 128),     │                │                        │
│                           │ (None, 128)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256)            │              0 │ bidirectional[0][1],   │
│                           │                        │                │ bidirectional[0][3]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 256)            │              0 │ bidirectional[0][2],   │
│ (Concatenate)             │                        │                │ bidirectional[0][4]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ (None, 320, 256)       │        525,312 │ bidirectional[0][0],   │
│                           │                        │                │ concatenate[0][0],     │
│                           │                        │                │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Attention_Layer           │ (None, 320, 256)       │              0 │ bidirectional[0][0],   │
│ (Attention)               │                        │                │ lstm_5[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concat_layer              │ (None, 320, 512)       │              0 │ lstm_5[0][0],          │
│ (Concatenate)             │                        │                │ Attention_Layer[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_2        │ (None, 320, 1)         │            513 │ concat_layer[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,706,497 (6.51 MB)

 Trainable params: 1,706,497 (6.51 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
t = trange(10, desc = 'Epoch', ncols = 90)
for epoch_i in t: # Running the model for given epoch numbers

    model_ed.fit(X_with_batch_size)


Epoch:   0%|                                                       | 0/10 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 837ms/step - accuracy: 0.5638 - loss: 0.9398


Epoch:  10%|████▋                                          | 1/10 [00:28<04:18, 28.68s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 794ms/step - accuracy: 0.8211 - loss: 0.4901


Epoch:  20%|█████████▍                                     | 2/10 [00:35<02:04, 15.60s/it]

...

Epoch:  80%|█████████████████████████████████████▌         | 8/10 [01:15<00:14,  7.31s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 840ms/step - accuracy: 0.8121 - loss: 0.4846


Epoch:  90%|██████████████████████████████████████████▎    | 9/10 [01:22<00:07,  7.09s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 813ms/step - accuracy: 0.8178 - loss: 0.4752


Epoch: 100%|██████████████████████████████████████████████| 10/10 [01:28<00:00,  8.87s/it]


In [ ]:
# modelEvaluatorED.evaluate_encoder_decoder_model(y, model_ed, "test_datasets")

In [ ]:
modelEvaluator.evaluate_model(y,model_ed,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Evaluate:  10%|████▍                                       | 1/10 [00:03<00:34,  3.78s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:04<00:14,  1.80s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:06<00:00,  2.57it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:06<00:00,  2.53it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  37  |  0.40909  | 0.40988 | 0.40946 |
|  3   |  0.56261  | 0.55487 | 0.55871 |
|  9   |  0.59903  | 0.59168 | 0.59533 |
|  40  |  0.54603  | 0.53939 | 0.54269 |
|  24  |  0.55897  | 0.55616 | 0.55754 |
|  34  |  0.54935  | 0.54380 | 0.54653 |
|  41  |  0.84075  | 0.82821 | 0.83440 |
|  49  |  0.60382  | 0.59540 | 0.59958 |
|  33  |  0.55028  | 0.54323 | 0.54673 |
|  38  |  0.51379  | 0.51647 | 0.51504 |
| mean |  0.52616  | 0.52067 | 0.52338 |
+------+-----------+---------+---------+


In [ ]:
# encoder decoder
import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model

encoder_inputs = Input(shape = (320, 1024))

encoder_BidirectionalLSTM = Bidirectional(LSTM(256, return_sequences = True, return_state = True)) # Encoder Layer
encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM(encoder_inputs)
sh = Concatenate()([fh, bh])
ch = Concatenate()([fc, bc])
encoder_states = [sh, ch]

decoder_LSTM = LSTM(512, return_sequences = True, dropout = 0.005, recurrent_dropout = 0.005) # Decoder Layer
decoder_out = decoder_LSTM(encoder_out, initial_state = encoder_states)

attn_layer = Attention(name="Attention_Layer") # Attention Layer
attn_out =  attn_layer([encoder_out, decoder_out])

decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_out, attn_out]) # Concatenating outputs of attention and decoder layer

dense = TimeDistributed(Dense(1, activation = 'tanh')) # Output layer
decoder_pred = dense(decoder_concat_input)

model_ed2 = Model(inputs = encoder_inputs, outputs = decoder_pred) # Model assembling

opt = tf.keras.optimizers.Adam(learning_rate=0.005, beta_1 = 0.7, beta_2 = 0.8)

model_ed2.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy']) # Model Compiling
model_ed2.summary()



Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 320, 1024)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ [(None, 320, 512),     │      2,623,488 │ input_layer_3[0][0]    │
│ (Bidirectional)           │ (None, 256), (None,    │                │                        │
│                           │ 256), (None, 256),     │                │                        │
│                           │ (None, 256)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 512)            │              0 │ bidirectional_1[0][1], │
│ (Concatenate)             │                        │                │ bidirectional_1[0][3]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 512)            │              0 │ bidirectional_1[0][2], │
│ (Concatenate)             │                        │                │ bidirectional_1[0][4]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ (None, 320, 512)       │      2,099,200 │ bidirectional_1[0][0], │
│                           │                        │                │ concatenate_2[0][0],   │
│                           │                        │                │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Attention_Layer           │ (None, 320, 512)       │              0 │ bidirectional_1[0][0], │
│ (Attention)               │                        │                │ lstm_7[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concat_layer              │ (None, 320, 1024)      │              0 │ lstm_7[0][0],          │
│ (Concatenate)             │                        │                │ Attention_Layer[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_3        │ (None, 320, 1)         │          1,025 │ concat_layer[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,723,713 (18.02 MB)

 Trainable params: 4,723,713 (18.02 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

for _ in trange(10, desc = 'Epoch', ncols = 90):
    model_ed2.fit(X_with_batch_size)

Epoch:   0%|                                                       | 0/10 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 833ms/step - accuracy: 0.3915 - loss: 9.6998


Epoch:  10%|████▋                                          | 1/10 [00:27<04:05, 27.29s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 838ms/step - accuracy: 0.1833 - loss: 13.0206


Epoch:  20%|█████████▍                                     | 2/10 [00:34<02:01, 15.22s/it]

...

Epoch:  80%|█████████████████████████████████████▌         | 8/10 [01:14<00:14,  7.16s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 813ms/step - accuracy: 0.1844 - loss: 13.0031


Epoch:  90%|██████████████████████████████████████████▎    | 9/10 [01:20<00:06,  6.98s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 827ms/step - accuracy: 0.1794 - loss: 13.0824


Epoch: 100%|██████████████████████████████████████████████| 10/10 [01:27<00:00,  8.74s/it]


In [ ]:
modelEvaluator.evaluate_model(y,model_ed2,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Evaluate:  10%|████▍                                       | 1/10 [00:03<00:34,  3.86s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:04<00:14,  1.83s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:06<00:00,  2.54it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:06<00:00,  2.52it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  37  |  0.24453  | 0.24578 | 0.24513 |
|  3   |  0.53211  | 0.52503 | 0.52854 |
|  9   |  0.60788  | 0.59693 | 0.60235 |
|  40  |  0.57296  | 0.56598 | 0.56945 |
|  24  |  0.52550  | 0.52600 | 0.52574 |
|  34  |  0.57371  | 0.55339 | 0.56334 |
|  41  |  0.80110  | 0.79198 | 0.79648 |
|  49  |  0.55472  | 0.54880 | 0.55174 |
|  33  |  0.52178  | 0.51458 | 0.51816 |
|  38  |  0.51379  | 0.51647 | 0.51504 |
| mean |  0.53082  | 0.52513 | 0.52793 |
+------+-----------+---------+---------+


In [ ]:
model_final = Sequential()
model_final.add(LSTM(128, return_sequences=True, input_shape=(320, 1024)))
model_final.add(LSTM(256, return_sequences=True))

model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))

model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))


model_final.add(Dense(256, activation='relu'))
model_final.add(Dense(128, activation='relu'))



model_final.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))


model_final.add(Dense(128, activation='relu'))
model_final.add(Dense(64, activation='relu'))


model_final.add(TimeDistributed(Dense(1, activation='sigmoid')))

In [ ]:
model_final.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_14 (LSTM)                       │ (None, 320, 128)            │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ (None, 320, 256)            │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_77 (Dense)                     │ (None, 320, 256)            │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_78 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_79 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_80 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_81 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_82 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_83 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_84 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_85 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_86 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_87 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_88 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_89 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_91 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_92 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_93 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_94 (Dense)                     │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_95 (Dense)                     │ (None, 320, 256)            │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 2,146,305 (8.19 MB)

 Trainable params: 2,146,305 (8.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_final.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'] )

In [ ]:
model_final.fit(X_with_batch_size, epochs=30)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 72ms/step - accuracy: 0.6117 - loss: 0.6814
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8212 - loss: 0.5418
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8122 - loss: 0.4983
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8190 - loss: 0.4770
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8182 - loss: 0.4752
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8211 - loss: 0.4731
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8195 - loss: 0.4725
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8176 - loss: 0.4756
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8227 - loss: 0.4676
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8131 - loss: 0.4824
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8186 - loss: 0.4738
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8161 - loss: 0.4776


In [ ]:
model_final2 = Sequential()
model_final2.add(LSTM(128, return_sequences=True, input_shape=(320, 1024)))
model_final2.add(LSTM(256, return_sequences=True))
model_final2.add(BatchNormalization()) # Add Batch Normalization
model_final2.add(Dense(256, activation='relu'))
model_final2.add(Dropout(0.2)) # Add Dropout for regularization
model_final2.add(Dense(128, activation='tanh')) # Try a different activation
model_final2.add(TimeDistributed(Dense(1, activation='sigmoid')))

# Compile the model with an adjusted learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.005)
model_final2.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_final2.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                       │ (None, 320, 128)            │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 320, 256)            │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 320, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_114 (Dense)                    │ (None, 320, 256)            │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 320, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_115 (Dense)                    │ (None, 320, 128)            │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_8 (TimeDistributed) │ (None, 320, 1)              │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,084,417 (4.14 MB)

 Trainable params: 1,083,905 (4.13 MB)

 Non-trainable params: 512 (2.00 KB)

In [ ]:
model_final2.fit(X_with_batch_size, epochs=30)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - accuracy: 0.7859 - loss: 0.5833
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8210 - loss: 0.4714
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8189 - loss: 0.4708
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8156 - loss: 0.4579
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8115 - loss: 0.4465
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8167 - loss: 0.4145
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8160 - loss: 0.3872
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8362 - loss: 0.3592
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8363 - loss: 0.3609
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.8511 - loss: 0.3459
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8528 - loss: 0.3377
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.8693 - loss: 0.3044


In [ ]:
modelEvaluator.evaluate_model(y,model_final,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Evaluate:  10%|████▍                                       | 1/10 [00:03<00:31,  3.53s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:03<00:12,  1.61s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:04<00:00,  4.18it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  4.04it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  37  |  0.36263  | 0.36033 | 0.36145 |
|  3   |  0.54901  | 0.54171 | 0.54534 |
|  9   |  0.57406  | 0.56755 | 0.57078 |
|  40  |  0.56046  | 0.55364 | 0.55703 |
|  24  |  0.52284  | 0.51710 | 0.51993 |
|  34  |  0.50147  | 0.50317 | 0.50229 |
|  41  |  0.82379  | 0.81355 | 0.81861 |
|  49  |  0.54477  | 0.53286 | 0.53874 |
|  33  |  0.57080  | 0.56063 | 0.56567 |
|  38  |  0.51379  | 0.51647 | 0.51504 |
| mean |  0.53513  | 0.52944 | 0.53224 |
+------+-----------+---------+---------+


In [ ]:
modelEvaluator.evaluate_model(y,model_final2,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Evaluate:  10%|████▍                                       | 1/10 [00:03<00:29,  3.28s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:03<00:12,  1.51s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:04<00:00,  4.31it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  4.18it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  37  |  0.40118  | 0.40174 | 0.40143 |
|  3   |  0.56261  | 0.55487 | 0.55871 |
|  9   |  0.58123  | 0.57071 | 0.57592 |
|  40  |  0.55872  | 0.55098 | 0.55482 |
|  24  |  0.49006  | 0.48600 | 0.48801 |
|  34  |  0.54935  | 0.54380 | 0.54653 |
|  41  |  0.76254  | 0.75197 | 0.75719 |
|  49  |  0.64448  | 0.62989 | 0.63709 |
|  33  |  0.52799  | 0.52123 | 0.52459 |
|  38  |  0.41203  | 0.40821 | 0.41003 |
| mean |  0.53745  | 0.53152 | 0.53444 |
+------+-----------+---------+---------+


In [ ]:
transformer_model = tf.keras.models.load_model('/content/drive/MyDrive/trainformer.h5')

In [ ]:
model_evaluator = ModelEvaluator()

In [ ]:
model_evaluator.evaluate_model(y,transformer_model,"test_datasets")

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Evaluate:  10%|████▍                                       | 1/10 [00:04<00:36,  4.02s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


Evaluate:  20%|████████▊                                   | 2/10 [00:04<00:14,  1.85s/it]

...

Evaluate:  80%|███████████████████████████████████▏        | 8/10 [00:06<00:00,  2.80it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step


Evaluate:  90%|███████████████████████████████████████▌    | 9/10 [00:06<00:00,  3.10it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  41  |  0.80723  | 0.79796 | 0.80254 |
|  1   |  0.67657  | 0.66507 | 0.67077 |
|  46  |  0.61081  | 0.60256 | 0.60666 |
|  50  |  0.55020  | 0.53454 | 0.54226 |
|  39  |  0.45914  | 0.45743 | 0.45827 |
|  11  |  0.68838  | 0.69077 | 0.68951 |
|  48  |  0.45656  | 0.43774 | 0.44691 |
|  31  |  0.57018  | 0.56285 | 0.56647 |
|  23  |  0.51869  | 0.51100 | 0.51481 |
|  12  |  0.62471  | 0.61559 | 0.62012 |
| mean |  0.59625  | 0.58755 | 0.59183 |
+------+-----------+---------+---------+
